In [ ]:
import numpy as np
import pandas as pd
import glob
import requests
import subprocess
import time
from dotenv import load_dotenv
import os

pd.set_option('display.max_columns', None)

################################################################################
#### Load Open FEC API Key 
################################################################################

# Define the path to .env file
env_path = "/Users/sarahmathey/Desktop/DSII/final_project/Credentials/.env"

# Load the environment variables
load_dotenv(env_path)

# Access the variables
my_secret = os.getenv("open_fec_key")

In [ ]:
################################################################################
#### Request Data from API 
################################################################################
# Step 1 is to grab a record of every F3X (Pac financial) filings for the 2024 cycle
# Base API URL
base_url = "https://api.open.fec.gov/v1/filings/"

# Number of filings you need to pull
total_filings = 90000
results_per_page = 100  # Max results per API call
total_calls = total_filings // results_per_page

# DataFrame to store results
all_filings = []

# Function to make API calls with pagination
def fetch_filings(page):
    params = {'cmte_type':'N%2C%20Q%2C%20V%2C%20W'
                , 'form_type':'F3X'
                , 'cycle':2024
                , 'most_recent':'true'
                , 'per_page': 100
                , 'api_key': my_secret
                , 'page': page}
    
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        return response.json()  # Return the JSON data from the API response
    else:
        print(f"Error fetching data for page {page}: {response.status_code}")
        return None

# Loop through pages and fetch filings
for page in range(1, total_calls + 1):
    print(f"Fetching page {page}...")

    # Fetch filings data for the current page using function defined above
    data = fetch_filings(page)
    
    if data:
        filings = data.get('results', [])
        all_filings.extend(filings)  # Add filings to the results list

        # Log progress every 10 pages
        if page % 10 == 0:
            print(f"Progress: {page}/{total_calls} pages fetched.")

    # Introduce a small delay to avoid rate-limiting (e.g., 1 second per request)
    time.sleep(5)

In [ ]:
# Convert the collected data into a DataFrame
filings_df = pd.DataFrame(all_filings)

In [ ]:
# Save the results to a CSV file 
filings_df.to_csv("fec_filings_2024.csv", index=False)
print("Filings data saved to 'fec_filings_2024.csv'")

In [ ]:
min(filings_df.coverage_end_date)

In [ ]:
filings_df.shape

In [ ]:
filings_df.file_number.nunique()

In [ ]:
################################################################################
#### Format Data from API Call
################################################################################
# Step 2 is to use the information gained from step 1, which includes some summary aggregation
# to pull lineitem details that will give us each itemized disbursement included in a given filing

#data = filing_response.json()
#filing_df = pd.DataFrame(data['results']) 

# Select and organize columns 
filing_df = filings_df[['committee_id','committee_type','committee_name','coverage_start_date',
                       'coverage_end_date','csv_url','cycle','document_description','fec_file_id',
                       'file_number','form_type','is_amended','most_recent','pages','receipt_date',
                       'report_type','report_type_full','report_year','treasurer_name','fec_url',
                       'cash_on_hand_beginning_period', 'cash_on_hand_end_period','debts_owed_by_committee',
                       'debts_owed_to_committee','total_communication_cost','total_disbursements',
                       'total_independent_expenditures','total_individual_contributions', 'total_receipts']]

In [ ]:
filing_df.head()

In [ ]:
filing_df.info()

We want to exclude the major fundraising conduits, ActBlue and WindRed since they have massive filings and will not be Scam PACS (though Scam PACs may use them) 

While PACs are the primary filers of F3X filings we want enforce this condition by filtering on committee type 

| Code |      Name |    Description |
|----------------|-----------|----------------|
| N|PAC - nonqualified| PACs that have not yet been in existence for six months and received contributions from 50 people and made contributions to five federal candidates. These committees have lower limits for their contributions to candidates.|
| O|Independent expenditure-only (Super PACs)|Political Committee that has filed a statement consistent with AO 2010-09 or AO 2010-11.|
| Q|PAC - qualified| PACs that have been in existence for six months and received contributions from 50 people and made contributions to five federal candidates|
| V|Hybrid PAC (with Non-Contribution Account) - Nonqualified| Political committees with non-contribution accounts|
| W|Hybrid PAC (with Non-Contribution Account) - Qualified| Political committees with non-contribution accounts|



[FEC Committee Type Documentation](https://www.fec.gov/campaign-finance-data/committee-type-code-descriptions/)


In [ ]:
conduits = ['C00401224' #ActBlue 
            ,'C00694323' #WinRed
            ]
pac_cmte_tps = ['N' ,'O' ,'Q' ,'V','W']

filings_to_download = filing_df[~filing_df['committee_id'].isin(conduits)]

filings_to_download = filings_to_download[filings_to_download['committee_type'].isin(pac_cmte_tps)]

# isolate unique, non-null filing_ids for use with FastFEC

filing_url_ids = filings_to_download[['fec_url','file_number']]

filing_url_ids=filing_url_ids[filing_url_ids.fec_url.notna()]

In [ ]:
filing_url_ids.shape

In [ ]:
dupe_filing_url_ids = filing_url_ids[filing_url_ids.duplicated()]

In [ ]:
dupe_filing_url_ids.file_number.head()

In [ ]:
filing_url_ids.drop_duplicates(inplace=True)

In [ ]:
os.chdir('/Users/sarahmathey/Desktop/DSII/final_project/data/raw')

In [ ]:
################################################################################
#### Download and process each filing using FastFec
################################################################################

# Directory to save output
output_dir = "output"

# Loop through each row in the DataFrame
for _, row in filing_url_ids.iterrows():
    fec_url = row["fec_url"]
    filing_id = str(row["file_number"]) 
    print(f"Processing {filing_id}...")

    # Use curl to download and pipe to fastfec
    command = f'curl -s {fec_url} | fastfec {filing_id}'
    subprocess.run(command, shell=True)

    # Get all files related to this filing
    filing_dir = os.path.join(output_dir, filing_id)  
    filing_files = glob.glob(os.path.join(filing_dir,'*'))

    # Check if there are any SB or SE files
    sb_files = [file for file in filing_files if os.path.basename(file).startswith("SB")]
    se_files = [file for file in filing_files if os.path.basename(file).startswith("SE")]

    if not sb_files and not se_files:
        # No SB or SE files? Delete all files for this filing
        print(f"No SB or SE files found for {filing_id}. Deleting all related files...")
        for file in filing_files:
            os.remove(file)
    else:
        # Otherwise, delete non-"SB" and non-"F3X" files
        for file in filing_files:
            filename = os.path.basename(file)
            if not (filename.startswith("SB") or filename.startswith("SE") or filename.startswith("F3X")):
                print(f"Deleting: {filename}")
                os.remove(file)

print("Processing complete!")


**Check to make sure the number of files in the output directory matches the number of file_numbers in filing_url_ids**

In [ ]:
directory = "/Users/sarahmathey/Desktop/DSII/final_project/data/raw/output"
file_list = [f for f in os.listdir(directory)]

In [ ]:
file_list = [pd.to_numeric(f) for f in file_list if f!='.DS_Store']

In [ ]:
filing_url_ids.file_number.nunique()

In [ ]:
len(file_list)

In [ ]:
unparsed_filings = filing_url_ids[~filing_url_ids['file_number'].isin(file_list)]

In [ ]:
unparsed_filings.file_number.nunique()

#### Delete all empty directories from filings with no relevant spending

In [ ]:
def delete_empty_directory(dir_path):
    """Deletes a directory if it is empty.

    Args:
        dir_path: The path to the directory.
    """
    # if the file path does not exist
    if not os.path.exists(dir_path):
        print(f"Directory not found: {dir_path}")
        return
    # if the file path is not a directory
    if not os.path.isdir(dir_path):
        print(f"Not a directory: {dir_path}")
        return
    # if the file path does not have any subfiles within
    if not os.listdir(dir_path):
        try:
            os.rmdir(dir_path)
            print(f"Directory deleted: {dir_path}")
        except OSError as e:
            print(f"Error deleting directory {dir_path}: {e}")
    else:
        print(f"Directory is not empty: {dir_path}")

In [ ]:
os.chdir('output')

In [ ]:
# list all the directories in output 
dirs = os.listdir()

In [ ]:
# create full file paths for all the directories in output 
full_dirs = []
root_dir = os.getcwd()
for dir in dirs:
    full_dir = os.path.join(root_dir, dir) 
    full_dirs.append(full_dir)

In [ ]:
# loop through all directories in output and run delete_empty_directory
for directory_path in full_dirs:
    delete_empty_directory(directory_path)

In [ ]:
# see how many directories are left
file_list_new = [f for f in os.listdir(directory)]

In [ ]:
len(file_list_new)

- 64281 filings were parsed
- 42717 had Schedule B or E line items 